# <center>Livrable Projet DATA SCIENCE</center>

### Contexte

L'entreprise TouNum est une entreprise de numérisation de documents. Elle prospose différents services dont la numérisation de base de document papier pour les entreprises clientes. TouNum veut optimiser et rendre intelligent ce processus de scanning en incluant des outils de Machine Learning. Le gain de temps serait important aux vues des nombreuses données que l'entreprise doit scanner et étiqueter.
Pour cela, TouNum fait appel à CESI pour réaliser cette prestation.

### Objectif

L'objectif est que l'équipe de data scientist de CESI réalise cette solution visant à analyser des photographies pour en déterminer une légende descriptive de manière automatique. Il faudra également améliorer la qualité des images scannées ayant des qualités variables (parfois floues, ou bruitées).

<img src="imageSrc/caption image.PNG"/>

### Enjeux

TouNum devait trier et étiqueter chaque document scanné. La solution délivré par CESI permet l'automatisation de ces tâches en faisant donc gagner un temps non négligeable. Elle va donc pouvoir réaliser plus de contrats et augmenter la satisfaction client.

### Contraintes techniques

L'implémentation des algorithmes doit être réaliser sur Python, notamment les librairies Scikit et TensorFlow. La librairie Pandas doit être utilisé pour manipuler le dataset et ImageIO pour le charger. NumPy et MatPlotLib seront nécessaire pour le calcul scientifique et la modélisation.

Le programme à livrer devra respecter le workflow suivant :

<img src="imageSrc/workflow.PNG"/>

#### Classification:

La classification d'image se fera à l'aide de réseaux de neurones. Cette dernière doit distinguer les photos d'un autre documents, tel que schémas, textes scannés, voir peintures.
TouNoum possède un dataset rempli d'images divers pour entrainer le réseau de neurones.

#### Prétraitement

Le prétraitement dois utiliser des filtres convolutifs afin d'améliorer la qualité. Il doit établir un compromis entre débruitage et affutage.

#### Captionning

Le Captionning devra légender automatiquement les images. Il utilisera deux techniques de Machine Learning : les réseaux de neurones convolutifs (CNN) pour prétraiter l'image en identifiant les zones d’intérêt, et les réseaux de neurones récurrents (RNN) pour générer les étiquettes. Il faudra être vigilant quant aux ressources RAM. Un dataset d'étiquetage classique est disponible pour l’apprentissage supervisé.

### Livrable

La solution doit sous forme de notebook Jupiter entièrement automatisé. Il doit être conçu pour être faciliter mis en production et maintenance.
Il faut démontrer la pertinence du modèle de manière rigoureuse et pédagogique.

#### Jalons

CESI devra dois rendre le prototype complet et fonctionnel du programme pour le 23 janvier. 
TouNum exige également 3 dates de rendu pour suivre la bonne avancé du projet.
<ul>
    <li>18/12/20 : Prétraitement d'image</li>
    <li>15/01/21 : Classification binaire</li>
    <li>20/01/21 : Captioning d'images</li>
    <li>22/01/21 : Démonstration </li>
</ul>


## Livrable 1 - Prétraitement (denoising/sharpening…)

Le but est de traiter un ensemble de photographies afin de les rendre mieux traitables par les algorithmes de Machine Learning. Il y a deux traitements à réaliser : le débruitage, et l’affutage. Vous devrez produire un notebook Jupyter explicitant ces étapes de prétraitement, et leurs performances. Ces algorithmes s’appuieront sur des notions assez simples autour des filtres de convolution, et les appliqueront pour améliorer la qualité de l’image. Il faudra notamment décider d’un compromis entre dé-bruitage et affutage.

Le notebook devra intégrer :
<ul>
    <li>Le code de chargement du fichier.</li>
    <li>Le code du débruitage sur un sous-ensemble d’images bruitées. Le code doit être accompagné d’explications.</li>
    <li>Le code de l’affutage sur un sous-ensembles d’images floutées. Le code doit être accompagné d’explications.</li>
    <li>
        Une étude de cas explicitant les compromis entre ces deux opérations. Cette partie du livrable doit inclure le bruitage d’images et montrer la perte de détails, ou l’affutage d’images et montrer l’apparition du bruit.
    </li>
</ul>

<b>Ce livrable est à fournir pour le 18/12/2020</b>

## *Importation des librairies utilisées*

In [ ]:
import os
import time
import random

# Check if imageio package is installed
try:
    import imageio
except ImportError:
    !pip install imageio
    
import imageio
import matplotlib.pyplot as plt
import numpy as np

# Check if cikit-image package is installed
try:
    import skimage
except ImportError:
    !pip install scikit-image

from skimage import io
from skimage.restoration import estimate_sigma

# Check if opencv-python package is installed
try:
    import cv2
except ImportError:
    !pip install opencv-python

import cv2

import threading
from queue import Queue
from multiprocessing import Pool

import pathlib

# Check if pandas package is installed
try:
    import pandas
except ImportError:
    !pip install pandas

import pandas as pd

import PIL
import PIL.Image

# Check if tensorflow package is installed
try:
    import tensorflow
except ImportError:
    !pip install tensorflow
# Check if tensorflow_datasets package is installed
try:
    import tensorflow_datasets
except ImportError:
    !pip install tensorflow_datasets    


import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers

# Check if tensorflow_datasets package is installed
try:
    import keras
except ImportError:
    !pip install keras    
from keras.preprocessing import image

# Livrable 3 
import json
import collections

from PIL import Image

# Lib to move images
import shutil

# Check if tqdm package is installed
try:
    import tqdm
except ImportError:
    !pip install -q tqdm

from tqdm import tqdm

from keras.callbacks import Callback,ModelCheckpoint
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
import keras.backend as K

## *Chemins physiques*

In [ ]:
blurry_dataset_path = "./Dataset/1/dataset/Blurry/"
noisy_dataset_path = "./Dataset/1/dataset/Noisy/"

deblured_dataset_path = "./Dataset/1/dataset/deblurred/"
denoised_dataset_path = "./Dataset/1/dataset/denoised/"

classification_dataset_path = "./Dataset/2/dataset/"
classification_model_path = "./Models/classification/"
classification_input_path = "./Dataset/2/input"
classification_output_path = "./Dataset/2/output"

captionning_dataset_path  = "./Dataset/3/dataset/train/"
captionning_annotation_path  = "./Dataset/3/dataset/annotation/"
captionning_model_path = "./Models/captionning/"

#### Méthodes utilisées

In [ ]:
# Chargement des images
def get_image(path, filename):
    return io.imread(path + filename)

# Sauvegarde des images
def save_image(path, filename, content):
    #Check if folder exists
    if not os.path.isdir(path):
        os.makedirs(path)
    imageio.imwrite(path + filename , content)

# Retrieve importants informations from data    
def get_metric_stat(pre_data, post_data):
    data = [
        np.array([min(pre_data), max(pre_data), np.median(pre_data), np.average(pre_data)]),
        np.array([min(post_data), max(post_data), np.median(post_data), np.average(post_data)])
    ]

    data_array = pd.DataFrame(data,
                              index = ["pre_processed", "post_processed"],
                              columns = ["Min value", "Max value", "Median value", "Average value"])
    print(data_array)

# Display all data in table
def get_list_data(data_tmp):
    data_array = pd.DataFrame(np.array(data_tmp),
                              columns = ["Name", "Before", "After"])
    print("\n")
    print(data_array)

## Défloutage de l'image

Pour le défloutage les images, on passe par le filtrage via **convolution**. L'opération de convolution consiste à faire glisser une autre matrice nommée filtre (de taille généralement inférieure à l'image traitée) tout le long de l'image et remplacer la valeur de chaque pixel de l'image par la somme du produit des éléments de cette matrice.

Les filtres d'amélioration de la netteté d'une image (ou filtres d'affutage de contours) permettent d'améliorer la qualité d'une image en accentuant les bords (ou en d'autres termes en accentuant les différences entres les pixels adjacents). L'affutage de contour consiste à prendre des différences.

Pour le défloutage des images, on utilise un **filtre Laplacien**.
Ce filtre nous permet d'affuter les images grâce à une fonction de convolution de la librairie opencv sur l'image récupérée.
La variante de filtre choisie nous permet sur le jeu de données fourni d'affuter les images suffisamment pour retirer le flou présent sans pour autant y ajouter de bruit.



### Explication de la matrice Laplacienne


L'approximation utilisée pour calculer le Laplacien est (si on prend la somme des l'approximation de la dérivée au sens des abscisses et des ordonnées) exprimée sous la forme $F(x+1,y)+F(x-1,y)+F(x,y+1)+F(x,y-1)-4F(x,y)$).

Ce qui nous donne la matrice suivante et sa variante prenant en compte les diagonales (en effet, il existe une multitude de variantes):

<img src="imageSrc/conv-laplacian.jpg"/>

La matrice de Laplace permet de mettre en évidence les contours d'une image comme on peut le voir selon l'image suivante:

<img src="imageSrc/laplacian_filtered_image.jpg"/>

On voit donc mieux les contours, mais on perds le sens de l'image de départ. Pour corriger cela, on ajoute donc la matrice identitaire (l'image de départ) sur l'image d'arrivé, d'où le coefficient 9 au lieux de 8 (cf image ci dessous):

<img src="imageSrc/laplace_conv_original.jpg"/>


In [ ]:
# Deblurring function
def remove_blur(img, high):
    kernel = []
    
    if high:
        # Creation of a Laplacian kernel to use for debluring
        kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
    else:
        kernel = np.array([[0,-1,0], [-1,5,-1], [0,-1,0]])
    
    # Convolution of the kernel with the image given in the function's parameter
    return cv2.filter2D(img, -1, kernel)

### Métrique d'évaluation du niveau de flou

Pour évaluer le niveau de flou, on utilise les contours Laplacien sur l'image (cf image ci dessus) et on évalue la variance de Laplace. Une variance faible indique qu'une faible plage de gris est utilisé (en d'autres termes, qu'il n'y a pas beaucoup de nuance utilisé, et donc qui se caractérise par du flou). Une variance élevé correspond donc à une large plage de gris utilisé (donc beaucoup plus de nuance disponible pour les détails). 

Il faut être vigilant à certaines images, tel qu'une photo du ciel, ou la variance sera faible même si cette dernière est net. L'indicateur devra alors être interprété en fonction du contexte.

<img src="imageSrc/Laplace_Variance.jpg"/>

In [ ]:
def get_blurry_indicator(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    fm = cv2.Laplacian(gray_image, cv2.CV_64F).var()
    return fm

In [ ]:
# Create the list of files to treat
listing = os.listdir(blurry_dataset_path)

### Execution

<ul>
    <li> Utilisation de threads pour optimiser le temps d'éxecution </li>
    <li> Calcul de la variance avant et après traitement </li>
</ul>

In [ ]:
# Thread execution
def process_fpath(name):
    path = blurry_dataset_path + name
    img = get_image(blurry_dataset_path,name)

     # Get initial Blur metric
    original_blur_metric = get_blurry_indicator(img)
    pre_processed_data.append(original_blur_metric)
    
    # Remove blur from the colored image image
    deblurred_img = remove_blur(img, high=True)

    # Get initial Blur metric
    processed_blur_metric = get_blurry_indicator(deblurred_img)
    post_processed_data.append(processed_blur_metric)
    
    #print("image " + name + " - initial : " + str(original_blur_metric)
     #   + " - processed : " + str(processed_blur_metric)
     #   + " - difference : " + str(processed_blur_metric - original_blur_metric)+"\n")

    data_preview_blurr.append([name, original_blur_metric, processed_blur_metric])

    # Saving Image
    save_image(deblured_dataset_path, name, deblurred_img)

# Loop on the list of file
threads = []
pre_processed_data = []
post_processed_data = []
data_preview_blurr = []

if __name__ == '__main__':
    for name in listing:
        #process_fpath(name)
        t = threading.Thread(target=process_fpath, args=(name,))
        threads.append(t)
        
    # Start them all
    for thread in threads:
        thread.start()

    # Wait for all to complete
    for thread in threads:
        thread.join()
    
    get_metric_stat(pre_processed_data, post_processed_data)
    get_list_data(data_preview_blurr)

In [ ]:
plt.figure(figsize=(24, 8))

def display_image_diff(originalPath, diffPath, filename=None):
    if not filename:
        # Get a random file from directory
        filename = random.choice(os.listdir(originalPath)) 
    
    plt.subplot(121)
    plt.imshow(get_image(originalPath, filename))
    plt.axis('off')
    plt.title("Original Image")
    
    # Corrected Image noise
    plt.subplot(122)
    plt.imshow(get_image(diffPath, filename))
    plt.axis('off')
    plt.title("Corrected Image")
    
# Filename MUST be the same for both directories    
display_image_diff(blurry_dataset_path, deblured_dataset_path)

# Débruitage
La capture d'un signal lumineux par un appareil photographique s'accompagne le plus souvent d'informations non désirées : le « bruit ». 
L'essentiel de ce « bruit » (des pixels trop clairs ou trop sombre en trop grand nombre ou de manière irrégulière, par exemple) est dû au capteur.

### Le débruitage par morceaux (par patchs)
Le débruitage par morceaux est une technique de débruitage d'image utilisant l'algorithme de réduction du bruit numérique appelé en Anglais "non-local means".
La méthode repose sur un principe simple, remplacer la couleur d'un pixel par une moyenne des couleurs de pixels similaires. Mais les pixels les plus similaires à un pixel donné n'ont aucune raison d'être proches. Il est donc nécessaire de scanner une vaste partie de l'image à la recherche de tous les pixels qui ressemblent vraiment au pixel que l'on veut débruiter.

### Pourquoi cette methode ?
Le résultat d'un tel filtrage permet d’amoindrir la perte de détails au sein de l'image, comparé aux filtres réalisant des moyennes localement tel que le filtre de Gauss ou le filtre de Wiener, le bruit généré par l'algorithme "non-local means" est plus proche du bruit blanc.

**Syntax:**
cv2.fastNlMeansDenoisingColored( P1, P2, float P3, float P4, int P5, int P6)

**Parameters:**
* P1 – Source Image Array
* P2 – Destination Image Array
* P3 – Size in pixels of the template patch that is used to compute weights.
* P4 – Size in pixels of the window that is used to compute a weighted average for the given pixel.
* P5 – Parameter regulating filter strength for luminance component.
* P6 – Same as above but for color components // Not used in a grayscale image.


In [ ]:
# Remove Noise function
def remove_noise(image, high):
    if high == 2:
        return cv2.fastNlMeansDenoisingColored(image, None, 10, 10, 7, 15)
    elif high == 1:
        return cv2.fastNlMeansDenoisingColored(image, None, 5, 10, 7, 15)
    else:
        return cv2.fastNlMeansDenoisingColored(image, None, 3, 3, 7, 15)

def estimate_noise(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return estimate_sigma(img)

In [ ]:
# Create the list of files to treat
listing = os.listdir(noisy_dataset_path)

In [ ]:
# Thread execution
def process_fpath(name):
    path = noisy_dataset_path + name
    img = get_image(noisy_dataset_path,name)
    
    # Get initial noise metric
    original_noise_metric = estimate_noise(img)
    pre_processed_data.append(original_noise_metric)
    
    denoised_img = remove_noise(img, high=2)
    
    # Get initial noise metric
    processed_noise_metric = estimate_noise(denoised_img)
    post_processed_data.append(processed_noise_metric)

    data_preview_denoised.append([name, original_noise_metric, processed_noise_metric])
    
    save_image(denoised_dataset_path, name, denoised_img)

# Loop on the list of file
threads = []
pre_processed_data = []
post_processed_data = []
data_preview_denoised = []

if __name__ == '__main__':
    for name in listing:
        #process_fpath(name)
        t = threading.Thread(target=process_fpath, args=(name,))
        threads.append(t)
        
    # Start them all
    for thread in threads:
        thread.start()

    # Wait for all to complete
    for thread in threads:
        thread.join()
    
    get_metric_stat(pre_processed_data, post_processed_data)
    get_list_data(data_preview_denoised)

In [ ]:
plt.figure(figsize=(24, 8))

# Filename MUST be the same for both directories    
display_image_diff(noisy_dataset_path, denoised_dataset_path)

# Optimisation entre Défloutage et Débruitage
Afin d'améliorer l'image au maximum, on peut effectuer les 2 operations à savoir, le traitement de bruit et le traitement de flou, sur une même image et utiliser nos métriques de performances pour automatiser l'application des traitements. Un autre point d'importance, l'ordre dans lequel on effectue les traitements a un impact sur la qualité de l'image.

### Procédé d'amelioration
Afin de determiner la qualité générale d'une image en termes de bruit et de flou, on utilise la moyenne des mesures effectuées précedement sur les differents jeux de tests. En fonction de la première mesure de l'image, on décide de commencer par un débruitage ou un défloutage. Ensuite, une nouvelle mesure est effectuée et est comparée à nouveau avec les mesures des jeux de données post traitement. 
Si l'image n'est toujours pas considérées viable, on effectue l'opération inverse à plus faible intensité pour essayer d'avoir le meilleur compromis. Pour ces traitements, on se limite à deux passages pour éviter de détériorer l'image à analyser et pour éviter des temps de traitements trop longs. 

### Résultats
Le résultat de ces tests a permis de demontrer que la meilleure solution consiste a commencer par effectuer un defloutage à forte intensité sur l'image puis, si c'est nécessaire, un debruitage à basse intensité. Cela représente le meilleur compromis au niveau de la qualité génerale de l'image.

L'affichage de ces test est disponible ci-dessous:

In [ ]:
#Choose noisy or blurry image
noisy = False

img = []
#if you want random testing
#img = get_image("./Dataset/Blurry/", random.choice(os.listdir("./Dataset/Blurry/")))

#image retrival
if noisy:
    img = get_image(noisy_dataset_path, "noisy_117.jpg")

else:
    img = get_image(blurry_dataset_path, "blurry_092.jpg")

#initial image measurements
initial_noise = estimate_noise(img)
initial_blur = get_blurry_indicator(img)

# print(initial_noise, initial_blur)

#image is blurry
if initial_blur < 3000:
    # high deblur of the image
    img_stage2 = remove_blur(img, high=False)
    
    #second image measurements
    second_noise = estimate_noise(img)
    second_blur = get_blurry_indicator(img)
    
    #image meets requirements in terms of noise
    if second_noise < 1:
        #displaying images
        plt.figure(figsize=(24, 8))
        plt.subplot(121)
        plt.imshow(img)
        plt.axis('off')
        plt.title("Original Image")
        plt.subplot(122)
        plt.imshow(img_stage2)
        plt.axis('off')
        plt.title("Corrected Image with only 1 deblur")
        plt.show()
    
    #image doesn't meets requirements in terms of noise
    else:
        #low denoise of the image
        img_stage3 = remove_noise(img_stage2, 0)
        
        img_stage3_low_noise = remove_noise(img_stage2, 0)
        
        #displaying images
        plt.figure(figsize=(24, 8))
        plt.subplot(131)
        plt.imshow(img)
        plt.axis('off')
        plt.title("Original Image")
        plt.subplot(132)
        plt.imshow(img_stage2)
        plt.axis('off')
        plt.title("Corrected Image with only 1 deblur")
        plt.subplot(133)
        plt.imshow(img_stage3)
        plt.axis('off')
        plt.title("Corrected Image with 1 deblur and 1 low denoise")
        plt.show()
        plt.figure(figsize=(24, 8))
        plt.subplot(141)
        plt.imshow(img_stage3_low_noise)
        plt.axis('off')
        plt.title("Corrected Image with 1 deblur and 1 low denoise")
        plt.show()
        
#if image is noisy      
if initial_noise > 1:
    # high denoise of the image
    img_stage2 = remove_noise(img, 2)
    
    img_stage2_low_noise = remove_noise(img, 1)
    
    #second image measurements
    second_noise = estimate_noise(img)
    second_blur = get_blurry_indicator(img)
    
    #image meets requirements in terms of blur
    if second_blur > 48000:
        #displaying images
        plt.figure(figsize=(24, 8))
        plt.subplot(121)
        plt.imshow(img)
        plt.axis('off')
        plt.title("Original Image")
        plt.subplot(122)
        plt.imshow(img_stage2)
        plt.axis('off')
        plt.title("Corrected Image with only 1 deblur")
        plt.subplot(123)
        plt.imshow(img_stage2_low_noise)
        plt.axis('off')
        plt.title("Corrected Image with only 1 deblur")
        plt.show()
    
    #image meets requirements in terms of blur
    else:
        #low deblur of the image
        img_stage3 = remove_blur(img_stage2, high=False)
        img_stage3_low_noise = remove_blur(img_stage2, high=False)
        
        #displaying images
        plt.figure(figsize=(24, 8))
        plt.subplot(131)
        plt.imshow(img)
        plt.axis('off')
        plt.title("Original Image")
        plt.subplot(132)
        plt.imshow(img_stage2)
        plt.axis('off')
        plt.title("Corrected Image with only 1 denoise")
        plt.subplot(133)
        plt.imshow(img_stage2_low_noise)
        plt.axis('off')
        plt.title("Corrected Image with only 1 medium denoise")
        plt.show()
        
        plt.figure(figsize=(24, 8))
        plt.subplot(131)
        plt.imshow(img_stage3)
        plt.axis('off')
        plt.title("Corrected Image with 1 denoise and 1 low deblur")
        plt.figure(figsize=(24, 8))
        plt.subplot(132)
        plt.imshow(img_stage3_low_noise)
        plt.axis('off')
        plt.title("Corrected Image with 1 medium denoise and 1 low deblur")
        plt.show()        

# Sources

## Image


## Défloutage

* https://www.pyimagesearch.com/2015/09/07/blur-detection-with-opencv/
* https://stackoverflow.com/questions/48319918/whats-the-theory-behind-computing-variance-of-an-image

## Debruitage
* https://docs.opencv.org/3.4/d5/d69/tutorial_py_non_local_means.html
* http://www.ipol.im/pub/art/2011/bcm_nlm/article.pdf


# Livrable 2 - Classification binaire

L’entreprise voulant automatiser la sélection de photos pour l’annotations, le livrable 2 devra fournir une méthode de classification se basant sur les réseaux de neurones afin de filtrer les images qui ne sont pas des photos du dataset de départ.

Le notebook devra intégrer :
<ul>
    <li>Le code TensorFlow ainsi qu’un schéma de l’architecture du réseau de neurones. Toutes les parties doivent être détaillée dans le notebook : les paramètre du réseau, la fonction de perte ainsi que l’algorithme d’optimisation utilisé pour l’entrainement.</li>
    <li>Un graphique contenant l’évolution de l’erreur d’entrainement ainsi que de l’erreur de test et l’évolution de l’accuracy pour ces deux datasets.</li>
    <li>L’analyse de ces résultats, notamment le compromis entre biais et variance (ou sur-apprentissage et sous-apprentissage).</li>
    <li>Une description des méthodes potentiellement utilisables pour améliorer les compromis biais/variance : technique de régularisation, drop out, early-stopping, …</li>
</ul>

Le but ultime est d’être capable de distinguer les photos parmi toutes ces images. Il est tout de même conseillé de commencer par les images les plus faciles à distinguer des photos, puis aller vers les dataset les plus difficiles à classifier (notamment, il y a dans le dataset peinture un certain nombre d’oeuvres au rendu assez réaliste, qui devraient vous poser problème).

<b>Ce livrable est à fournir pour le 18/01/2021</b>

In [ ]:
#basics checks for image classifications
# print("executing tensorflow version " + tf.__version__)

if (len(tf.config.experimental.list_physical_devices('GPU')) == 1):
    print("GPU is detected")
else :
    print("GPU isn't detected")

## Paramétres

**Batch Size** : 

le nombre d'échantillons qui seront propagés à travers le réseau a chaque itération.

Le Batch size utilisé classiquement est de 32 mais nous l'avons reduit a 16, il a été empiriquement démontré que l'utilisation de lots plus petits permettait une convergence plus rapide vers de "bonnes" solutions. Cela s'explique notament par le fait que des tailles de lot plus petites permettent au modèle de commencer à apprendre avant de voir toutes les données.

**La taille de l'image** :

Reduit a 200*200 dans un soucis de preservation de la memoire vive.

**Validation Split** : 

Il s'agit du paramètre qui spécifie la taille des données de formation qui seront utilisées pour la validation. C'est une valeur flottante entre 0 et 1. Les données de validation ne sont pas utilisées pour la formation, mais pour évaluer la perte et la précision.

Nous l'avons fixé a 20% car dans de nombreux article et autre exemple, nous avons vu que la valeur etait generalement fixé a ce seuil.

**Epochs** :

C'est un hyperparamètre qui définit le nombre de fois que l'algorithme d'apprentissage fonctionnera sur l'ensemble des données de formation.

Dans notre test final, nous avons mis 20 epochs car la validation accuracy n'augmente plus.



In [ ]:
#Parameters for the dataset (amount of images per batch, image resolution and training percentage)
batch_size = 16
img_height = 200
img_width = 200
validation_split = 0.2
epochs=20
classes = ['Photo', 'Other']

## Le Modèle

<img src="imageSrc\ShemaNeural.PNG">

Le modèle se compose de trois blocs de convolution avec une couche de pool maximum dans chacun d'eux.

Il y a une couche entièrement connectée avec 128 unités sur le dessus qui est activée par une fonction d'activation "relu". 

Ce modèle basique trouvé sur tensorflow tutoriel a eté modifié via les parametres et l'ajout de layers avec nos nombreux tests.

**Le kernel** : 

Nous avons eu le choix entre plusieurs kernel 1x1, 2x2, 3x3, 7x7, 9x9, etc...  
L'une des raisons pour lesquelles nous avons préfère les noyaux de petite taille aux réseaux entièrement connectés est qu'ils réduisent les coûts de calcul et le partage des      poids, ce qui conduit en fin de compte à des poids moins importants.

1x1 a été éliminé car les caractéristiques extraites seront finement granulées et locales donnant aucune information provenant des pixels voisins.

Et nous avons eliminé tout les kernel de nombre pair car il n'y a pas de valeur central et cela creer une perte de precision.

**Relu**:

 "Rectified Linear Unit" est une fonction d'activation linéaire par morceaux, c'est la methode la plus utilisé de nos jours. Son avantage réside sur le fait qu'elle remplace toute valeur d'entrée négative par 0 et toute valeur positive par elle même.

Nous avons choisi de rajouter une convolution au model de base dans un besoins de précision, cela a amelioré nos resultats.




In [ ]:
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

def generate_model():
    #generation of the training dataset
    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
      classification_dataset_path,
      validation_split=validation_split,
      subset="training",
      seed=123,
      image_size=(img_height, img_width),
      batch_size=batch_size)

    #generation of the validation dataset
    val_ds = tf.keras.preprocessing.image_dataset_from_directory(
      classification_dataset_path,
      validation_split=validation_split,
      subset="validation",
      seed=123,
      image_size=(img_height, img_width),
      batch_size=batch_size)
    
    #retrieve the amount of classes for the model
    num_classes = len(train_ds.class_names)
    print("Classes found : " + str(num_classes))
    print(train_ds.class_names)

    #Allow for perfomance compilation times by preventing IO bottleneck on disks while compiling the model
    AUTOTUNE = tf.data.experimental.AUTOTUNE
    train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
    val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
    
    #Structure of the neural network
    model = tf.keras.Sequential([
      layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
      layers.Conv2D(16, 3, activation='relu'),
      layers.MaxPooling2D(),
      layers.Conv2D(32, 3, activation='relu'),
      layers.MaxPooling2D(),
      layers.Conv2D(64, 3, activation='relu'),
      layers.MaxPooling2D(),
      layers.Dense(128, activation='relu'),
      layers.GlobalAveragePooling2D(),
      layers.Dropout(0.2),
      layers.Dense(2)
    ])
    
    #display neural network structure
    model.summary()

    #compile the model
    model.compile(
        optimizer='adam',
        loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy', get_f1])
    
    #amount of training and fitting
    history = model.fit(
      train_ds,
      validation_data=val_ds,
      epochs=epochs
    )
    
    #display statitics over training accuracy
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs_range = range(epochs)

    plt.figure(figsize=(8, 8))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.show()
    
    return model

def classify_image(model, classes, impath):
    #load disj image
    img = image.load_img((impath), target_size=(img_height, img_width))
    img  = image.img_to_array(img)
    img  = img.reshape((1,) + img.shape)

    #use model to predict classe
    prediction = model.predict(img)
    score = tf.nn.softmax(prediction[0])
    #     print(
    #         "This image most likely belongs to {} with a {:.2f} percent confidence."
    #         .format(classes[np.argmax(score)], 100 * np.max(score))
    #     )
    #return clas and percentage of confidence
    return [classes[np.argmax(score)], 100 * np.max(score)]

In [ ]:
model = generate_model()

In [ ]:
def get_model(newModel=False):
    # Checking if the Models folder is empty
    if newModel == True: 
        model = generate_model()
        tf.keras.models.save_model(model, classification_model_path)
        return model    
    # Or not
    else:
        return tf.keras.models.load_model(classification_model_path)

model = get_model()

In [ ]:
# Seuil de confiance à partir duquel les images 'Photo' sont acceptées
confidence_threshold = 90

# Classify images and moved "Photo" to the output folder 
for pictures in os.listdir(classification_input_path):
    res_classe, res_score = classify_image(model, classes, os.path.join(classification_input_path,pictures))
    print(pictures, res_classe, res_score, '%')

    if (res_classe == 'Photo' and res_score > confidence_threshold):
        picture_path = classification_input_path + "/" + pictures
        shutil.copy2(picture_path, classification_output_path) 

# Single image test
#img_name = 'photo_0001.jpg'
#result = classify_image(model, classes, './Dataset/2/input/'+img_name)
#print(result)

## Résultat

Dans un premier temps nous avons créer un modele qui prend en input les 5 classes et rend 5 classes en sortie. Ce modele permet donc d'identifier le type d'image en entrée. 


<img src="imageSrc\resultLiv2.png">

**Premier test**

Premier résultat : 

Voici les résultats obtenus apres la compilation du modele avec les hyperparametres suivant :

Epochs : 20

Resolution de l'image : 200*200

Bach size : 16

Kernel : 3*3

Notre modele n'est pas optimal :
- Le training and Validation accuracy nous indique des erreurs de validation, une faible précision de validation par rapport à la précision de formation, ce qui indique un fort overfitting. 

- Le training and Validation loss quant a lui nous montre que notre modele a une augmentation des erreurs sur la validation notre modele a besoins d'etre modifié.



<img src="imageSrc\2resultLiv2.png">

**Second test**

Epochs : 20

Resolution de l'image : 200*200

Bach size : 16

Kernel : 3*3

Droupout : 50% 

**Dropout** est une technique où des neurones choisis au hasard sont ignorés pendant l'entraînement. Ils sont "abandonnés" au hasard. Cela signifie que leur contribution à l'activation des neurones en aval est temporairement supprimée lors du passage et toute mise à jour du poids n'est pas appliquée au neurone.

Notre modele n'est pas optimal :
- Le training and Validation accuracy : la courbe ressemble un peu plus a ce que l'on veut voir mais la precision a baissé.

- Le training and Validation loss quand a lui nous montre que notre modele a un bon taux d'apprentissage et la validation loss est decendant.



<img src="imageSrc\3graphliv2.png">

**Troisieme Test**

Nous avons changé de modele pour un modele binaire, nous prenons en entré seulement les photos et autres type d'image. En sorti, nous avons l'identification des images en tant que photo ou non.
Nous avons reduit aussi les epochs pour eviter le surapprentissage du reseau de neurone.

Epochs : 8

Resolution de l'image : 200*200

Bach size : 16

Kernel : 3*3

Droupout : 50% 




<img src="imageSrc\4resultliv2.png">

**Quatrieme Test réussite**

Nous avons changé de modèle pour un modèle binaire, nous prenons en entrée seulement les photos et autres types d'image. En sortit, nous avons l'identification des images en tant que photo ou non.
Nous avons réduit aussi les epochs pour éviter le surapprentissage du réseau de neurones.

Epochs : 20

Resolution de l'image : 200*200

Bach size : 16, 32, 64

Kernel : 3*3

Droupout : 5% 

Nous avons un résultat très satisfaisant montrant une précision sur la validation suivant bien le training accuracy et qui atteint un degret de précision haut, la perte sur la validation baisse aussi significativement en parallèle de la perte sur le training.




In [ ]:
#test autre model

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
      classification_dataset_path,
      validation_split=validation_split,
      subset="training",
      seed=123,
      image_size=(img_height, img_width),
      batch_size=batch_size)

    #generation of the validation dataset
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
      classification_dataset_path,
      validation_split=validation_split,
      subset="validation",
      seed=123,
      image_size=(img_height, img_width),
      batch_size=batch_size)

def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)
    
    data_augmentation = keras.Sequential(
        [
            layers.experimental.preprocessing.RandomFlip("horizontal"),
            layers.experimental.preprocessing.RandomRotation(0.1),
        ])
    
    # Image augmentation block
    x = data_augmentation(inputs)

    # Entry block
    x = layers.experimental.preprocessing.Rescaling(1.0 / 255)(x)
    x = layers.Conv2D(32, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.Conv2D(64, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [128, 256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes

    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs)


model = make_model(input_shape= (img_height,img_width) + (3,), num_classes=2)
keras.utils.plot_model(model, show_shapes=True)

model.summary()

epochs = 20

model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)
model.fit(
    train_ds, epochs=epochs, callbacks=callbacks, validation_data=val_ds,
)